# Environmental footprint of digital content consumption - Project setup

In [1]:
import pandas as pd
import brightway2 as bw
from bw2io import ExcelLCIAImporter
from premise import *
from pathlib import Path

## Create new BW project and import databases

In [2]:
PROJECT_NAME = 'iri' # <- insert your project name here
bw.projects.set_current(PROJECT_NAME)

**Import biosphere database**

In [3]:
if 'biosphere3' in bw.databases:
    print('Biosphere database already installed')
else:
    bw.bw2setup()

Biosphere database already installed


**Import ecoinvent database**

Note that the results presented in the scientific article were produced with ecoinvent v3.8 cut-off system model

In [4]:
ECOINVENT_DIR = r'.../ei38/datasets' # <- insert the path to ecoinvent 3.8 here
ECOINVENT_NAME = 'ecoinvent 3.8 cutoff'

if ECOINVENT_NAME in bw.databases:
    print('Ecoinvent database already installed')
else:
    ei38 = bw.SingleOutputEcospold2Importer(ECOINVENT_DIR, ECOINVENT_NAME)
    ei38.apply_strategies()
    ei38.statistics()
    ei38.write_database()

Ecoinvent database already installed


**Create prospective databases with premise**

The use of default IAM scenarios included in *premise* requires a decryption key to be requested from the library maintainers. For further details on *premise* see: https://github.com/polca/premise

In [3]:
DECRYPTION_KEY = 'XXXXXXXXXXXXXXXXXXXX' # <- insert decryption key here

In [10]:
ei_future = NewDatabase(
                scenarios=[{"model": "image", "pathway":"SSP2-Base", "year":2030},
                           {"model": "image", "pathway":"SSP2-Base", "year":2040},
                           {"model": "image", "pathway":"SSP2-Base", "year":2050},

                           {"model": "image", "pathway":"SSP2-RCP26", "year":2030},
                           {"model": "image", "pathway":"SSP2-RCP26", "year":2040},
                           {"model": "image", "pathway":"SSP2-RCP26", "year":2050},

                           {"model": "image", "pathway":"SSP2-RCP19", "year":2030},
                           {"model": "image", "pathway":"SSP2-RCP19", "year":2040},
                           {"model": "image", "pathway":"SSP2-RCP19", "year":2050},
                          ],
                source_db="ecoinvent 3.8 cutoff",
                source_version="3.8",
                key=DECRYPTION_KEY)

premise v.(1, 4, 2)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [11]:
ei_future.update_all()

`update_all()` will skip the following steps:
update_two_wheelers(), update_cars(), and update_buses()
If you want to update these steps, please run them separately afterwards.

////////////////// MEDIUM AND HEAVY DUTY TRUCKS ////////////////////
Extracted 1 worksheets in 6.86 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Done!
Extracted 1 worksheets in 10.62 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Done!
Extracted 1 worksheets in 12.33 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Done!

/////////////////////////// ELECTRICITY ////////////////////////////
Update natural gas extraction datasets.
Update efficiency of solar PV.
Log of changes in photovoltaics efficiencies saved in C:\Users\istrateir\Anaconda3\envs\iri\Lib\site-packages\premise\data/logs
Create biomass markets.
Repla

In [12]:
ei_future.write_db_to_brightway(name=["ecoinvent 3.8 cutoff, IMAGE SSP2-Base, 2030",
                                      "ecoinvent 3.8 cutoff, IMAGE SSP2-Base, 2040",
                                      "ecoinvent 3.8 cutoff, IMAGE SSP2-Base, 2050",

                                      "ecoinvent 3.8 cutoff, IMAGE SSP2-RCP26, 2030",
                                      "ecoinvent 3.8 cutoff, IMAGE SSP2-RCP26, 2040",
                                      "ecoinvent 3.8 cutoff, IMAGE SSP2-RCP26, 2050",

                                      "ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2030",
                                      "ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2040",
                                      "ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2050",
                                     ])

Write new database(s) to Brightway2.
Prepare database 1.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:50


Title: Writing activities to SQLite3 database:
  Started: 04/24/2023 11:53:41
  Finished: 04/24/2023 11:55:32
  Total time elapsed: 00:01:50
  CPU %: 99.40
  Memory %: 22.83
Created database: ecoinvent 3.8 cutoff, IMAGE SSP2-Base, 2030
Prepare database 2.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:14


Title: Writing activities to SQLite3 database:
  Started: 04/24/2023 12:13:19
  Finished: 04/24/2023 12:14:33
  Total time elapsed: 00:01:14
  CPU %: 99.10
  Memory %: 21.10
Created database: ecoinvent 3.8 cutoff, IMAGE SSP2-Base, 2040
Prepare database 3.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:14


Title: Writing activities to SQLite3 database:
  Started: 04/24/2023 12:31:27
  Finished: 04/24/2023 12:32:42
  Total time elapsed: 00:01:14
  CPU %: 99.00
  Memory %: 24.02
Created database: ecoinvent 3.8 cutoff, IMAGE SSP2-Base, 2050


## Create/update LCIA methods

Here we implement some new methods that are not available in the biosphere database compatible with Ecoinvent 3.8, including:

- IPCC 2021 global warming potentials (GWPs) adapted to premise
- EF 3.1 for ecotoxicity and human toxicity
- LANCA v2.5 for land use soil erosion (compatible with the carrying capacity)

The following data files are required (to be obtained from the corresponding sources and placed in the data container):

- Methods implementation in Ecoinvent 3.9: "LCIA Implementation 3.9.1.xlsx" (available at https://ecoinvent.org/)
- Because the name of some elementary flows change from Ecoinvent 3.8 to 3.9.1, the change report file is also required: "Change Report Annex v3.8 - v3.9.xlsx" (available at https://ecoinvent.org/)
- 100-year GWPs adapted to premise: "lcia_gwp_100a_w_bio.xlsx" (available at https://github.com/polca/premise_gwp)
- Caracterization factors from LANCA v2.5 (data file provided in this repository)

In [5]:
# Import LCIA implementation data
METHODS_DIR = Path(r"..\data\LCIA Implementation 3.9.1.xlsx")
CHANGE_REPORT_DIR = Path(r'..\data\Change Report Annex v3.8 - v3.9.xlsx')
PREMISE_GWP_DIR = Path(r'..\data\lcia_gwp_100a_w_bio.xlsx')

lcia_ei_391 = pd.read_excel(METHODS_DIR, sheet_name='CFs')
lcia_change_ei39vs38 = pd.read_excel(CHANGE_REPORT_DIR, sheet_name='LCIA Changes')
premise_gwp = pd.read_excel(PREMISE_GWP_DIR)

In [6]:
# Map biosphere flows
map_bio_keys = {}
for ef in bw.Database('biosphere3'):
    map_bio_keys[(ef['name'], ef['categories'])] = ef.key

**Climate change: 100-year GWPs from IPCC 2021 adapted to premise**

Since prospective databases generated with *premise* may involve inventories containing
net negative emission technologies (NETs), it is highly recommended to account for
biogenic CO2 flows as well as CO2 captured from the atmosphere. For further details see: https://github.com/polca/premise_gwp

Premise GWPs are based on IPCC2013. Here we update the method with GWPs from IPCC2021

In [7]:
# Get 100-year GWPs from IPCC 2021 as implemented in ecoinvent 3.9.1
ipcc2021_gwp_100a = lcia_ei_391[(lcia_ei_391['Method'] == 'IPCC 2021') & 
                                (lcia_ei_391['Category'] == 'climate change: including SLCFs') & 
                                (lcia_ei_391['Indicator'] == 'global warming potential (GWP100)')]

# Adapt GWPs from IPCC 2021 to premise GWPs:
premise_ipcc2021_gwp_100a = dict()
for index, row in premise_gwp.iterrows():
    premise_ipcc2021_gwp_100a.update({(row['name'], row['categories']): row['amount']})

    # Update existing and add new characterization factors based on IPCC2021
    for index, row in ipcc2021_gwp_100a.iterrows():
        ef_info = (row['Name'], row['Compartment'] + '::' + row['Subcompartment'])
        # Update existing elementary flows:
        if ef_info in premise_ipcc2021_gwp_100a:
            premise_ipcc2021_gwp_100a[ef_info] = row['CF']
        else:
           premise_ipcc2021_gwp_100a.update({ef_info: row['CF']})

    # Export the modified IPCC method:
    premise_ipcc2021_df = pd.DataFrame(premise_ipcc2021_gwp_100a, index=['amount']).T.reset_index()
    premise_ipcc2021_df.columns = ['name', 'categories', 'amount']
    premise_ipcc2021_df.to_excel(Path(r"..\data\LCIA_gwp_100a_ipcc2021_premise.xlsx"), index=False)

In [8]:
# Write method:
metadata =(
           ("IPCC 2021", "climate change: including SLCFs", "GWP 100a, incl. H and bio CO2"),
            "kg CO2-Eq",
            "IPCC 20121, with CFs for hydrogen and biogenic CO2 flows",
            Path(r"..\data\LCIA_gwp_100a_ipcc2021_premise.xlsx"),
          )

ipcc2021_method = ExcelLCIAImporter(filepath=metadata[-1], name=metadata[0], unit=metadata[1], description=metadata[2])
ipcc2021_method.apply_strategies()
ipcc2021_method.drop_unlinked()
assert len(list(ipcc2021_method.unlinked)) == 0
ipcc2021_method.write_methods(overwrite=True, verbose=True)

Applying strategy: csv_restore_tuples
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 8 strategies in 0.11 seconds
Applying strategy: drop_unlinked_cfs
Applied 1 strategies in 0.00 seconds
Wrote 1 LCIA methods with 249 characterization factors


**Life cycle CO2 emissions**

The carrying capacity for climate change is defined based on the remaining carbon budget. This requires quantifying the life cycle CO2 emissions. We create a new method to quantify this emissions

In [9]:
# Use the new IPCC2021 method to retrieve CO2 flows and characterization factors
ipcc_2021_method = bw.Method(('IPCC 2021', 'climate change: including SLCFs', 'GWP 100a, incl. H and bio CO2'))
gwp = ipcc_2021_method.load()
co2_cf = [i for i in gwp if 'Carbon dioxide' in bw.Database('biosphere3').get(i[0][1])['name']]

In [10]:
# Write method
metadata = {"unit": 'kg CO2'}
co2_method = bw.Method(('IPCC 2021', 'climate change: CO2 emissions'))
co2_method.register(**metadata)
co2_method.write(co2_cf)
co2_method.process()

**Land use: LANCA method for erosion potential due to land occupation and transformation (to/from)**

In [11]:
metadata = (
            ("LANCA v2.5 - land use", "erosion potential"),
            "kg soil loss",
            "LANCA v2.5 - Characterization Factors for Erosion Potential due to land occupation and transformation (to/from) \
            Available at: https://www.bookshop.fraunhofer.de/buch/LANCA/244600",
            Path(r"..\data\Soil-Erosion-Potential_CFs_LANCA_v2.5.xlsx")
            )

lanca_method = bw.ExcelLCIAImporter(filepath=metadata[-1], name=metadata[0], unit=metadata[1], description=metadata[2])
lanca_method.apply_strategies()
lanca_method.drop_unlinked()
lanca_method.write_methods(overwrite=True, verbose=True)

Applying strategy: csv_restore_tuples
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 8 strategies in 0.16 seconds
Applying strategy: drop_unlinked_cfs
Applied 1 strategies in 0.00 seconds
Wrote 1 LCIA methods with 135 characterization factors


**Human toxicity and ecotoxicity: Environmental Footprint 3.1**

In [12]:
# Biosphere flows and CFs for toxicity and human toxicity categories in EF3.1 as implemented in Ecoinvent 3.9.1
toxicity_categories = ['ecotoxicity: freshwater', 'human toxicity: carcinogenic', 'human toxicity: non-carcinogenic']
ef31_data = lcia_ei_391[(lcia_ei_391['Method'] == 'EF v3.1') & (lcia_ei_391['Category'].isin(toxicity_categories))]

In [13]:
# Biosphere database in ecoinvent 3.9 includes more elementary flows and the name of some flows changes with respect to 3.8
# We use the ecoinvent change report file to identify such differences
ef30_ei38vsei39 = lcia_change_ei39vs38[(lcia_change_ei39vs38['Method'] == 'EF v3.0') & (lcia_change_ei39vs38['Category - 3.8'].isin(toxicity_categories))]

# Drop flows that are introduced in Ecoinvent 3.9; these flows are not considered here
ef30_ei38vsei39 = ef30_ei38vsei39[ef30_ei38vsei39['Name - 3.8'].notna()]

# Identify biosphere flows named differently in ecoinvent 3.9 vs 3.8
bio_ei38_vs_ei39 = dict()
for index, row in ef30_ei38vsei39.iterrows():
    if row['Name - 3.8'] != row['Name - 3.9']:
        bio_ei38_vs_ei39.update({(row['Name - 3.9'], row['Compartment - 3.9'], row['Subcompartment - 3.9']): (row['Name - 3.8'], row['Compartment - 3.8'], row['Subcompartment - 3.8'])})

In [14]:
# List of elementary flows in biosphere in Ecoinvent 3.8
flows_in_bio = []
for flow in bw.Database('biosphere3'):
    flow_info = (flow['name'], flow['categories'][0], flow['categories'][1] if len(flow['categories']) > 1 else 'unspecified')
    flows_in_bio.append(flow_info)

In [15]:
cf_method_implementation = []
flows_not_in_bio = []
for index, row in ef31_data.iterrows():
    flow = (row['Name'], row['Compartment'], row['Subcompartment'])
    if flow in bio_ei38_vs_ei39:
        flow = bio_ei38_vs_ei39[flow]

    if flow in flows_in_bio:
        categories = (flow[1], flow[2]) if flow[2] != 'unspecified' else (flow[1],)
        cf_method_implementation.append((row['Category'], flow[0], categories, row['CF']))
    else:
        flows_not_in_bio.append(flow)

flows_not_in_bio = list(set(flows_not_in_bio))
print('Number of flows in biosphere:', len(flows_in_bio))
print('Number of flows not in biosphere:', len(flows_not_in_bio))

Number of flows in biosphere: 4427
Number of flows not in biosphere: 379


In [16]:
methods_name = {
                toxicity_categories[0]: ('EF v3.1 adapted to biosphere ei38', 'ecotoxicity: freshwater', 'comparative toxic unit for ecosystems (CTUe)'),
                toxicity_categories[1]: ('EF v3.1 adapted to biosphere ei38', 'human toxicity: carcinogenic', 'comparative toxic unit for human (CTUh)'),
                toxicity_categories[2]: ('EF v3.1 adapted to biosphere ei38', 'human toxicity: non-carcinogenic', 'comparative toxic unit for human (CTUh)')
               }

metadata = {toxicity_categories[0]: {"unit": 'CTUe'},
            toxicity_categories[1]: {"unit": 'CTUh'},
            toxicity_categories[2]: {"unit": 'CTUh'}
           }

for m in toxicity_categories:
    method = bw.Method(methods_name[m])
    method.register(**metadata[m])
    cf_data = [(map_bio_keys[(i[1], i[2])], i[3]) for i in cf_method_implementation if i[0] == m]
    method.write(cf_data)
    method.process()